In [1]:
import sys

sys.path.append("..")

In [2]:
import pandas as pd

from utils.timer import timer
from utils.excels import each_row, save_excel

In [100]:
import os

Bytes_Per_Sec = 201953523 / 333.0

@timer
def read_excel(file, name=None, rows=None):
    """
    读取Excel
    
    默认读取所有sheet, 可以指定
    默认读取所有行, 可以指定: 指定小批量行数并不会节约时间
    读取200MB约350s
    """
    file_bytes = os.path.getsize(file)
    print("predict cost: %.5fs" % (file_bytes / Bytes_Per_Sec))
    return pd.read_excel(file, nrows=rows, sheet_name=name)


In [12]:
abs_path = '/Users/tobin/Downloads/douyin_20190519/target_video_comments/'


## 用户处理

In [13]:
user_file='VIVO抖音项目-用户信息-0606.xlsx'

user_sheet = read_excel(abs_path + user_file, "模板")
print(user_sheet.shape)

# start at 1559904886.35080
# end at 1559904886.65185
total cost: 0.30105
(1389, 29)


In [24]:
user_tags = {}

def user_parser(idx, key, row):
    user_tag = {}
    
    uid = row["uid"]
    name = row["user_name"]
    nick = row["register_id"]
    fans = row["fans_num"]
    
    user_tag["uid"] = uid
    user_tag["name"] = name
    user_tag["fans"] = fans
    user_tag["register_id"] = nick
    
    user_tags[uid] = user_tag
    
    return True
    
each_row(user_sheet, user_parser, 1000)

# start at 1559905217.90615
current time: 1559905218.026327
current idx: 1000
# end at 1559905218.06156
total cost: 0.15541


In [25]:
print(len(user_tags))
for k, v in user_tags.items():
    print(k, v)
    break

1389
100003454600 {'uid': 100003454600, 'name': '极好金服', 'fans': 288797, 'register_id': 'jihaojinfu'}


## 视频处理

In [36]:
video_file = 'analysis_task_20190606230755_761_93_video.xlsx'

video_sheet = read_excel(abs_path + video_file, "sheet0")
print(video_sheet.shape)

# start at 1559905903.46626
# end at 1559905903.71271
total cost: 0.24645
(760, 30)


In [107]:
brand_file = "VIVO品牌码表.xlsx"

brand_sheet = read_excel(abs_path + brand_file, "Sheet1")
print(brand_sheet.shape)

# start at 1559909366.56457
predict cost: 0.02142s
# end at 1559909366.57867
total cost: 0.01410
(56, 5)


In [108]:
brandKB = {}

def brand_parser(idx, key, row):
    brand_name = row['品牌'].strip()
    if brand_name not in brandKB:
        brandKB[brand_name] = set()
    
    if type(row['品牌产品']) is str:
        brand_type_name = row['品牌产品'].strip()
        if brand_type_name:
            brandKB[brand_name].add(brand_type_name)
    
    brand_keyword_str = row['关键词'].strip()
    if brand_keyword_str.startswith('#'):
        brandKB[brand_name].add(brand_keyword_str)
    else:
        for kw in brand_keyword_str.split(','):
            if type(kw) is str:
                brandKB[brand_name].add(kw)
    
    return True
                
each_row(brand_sheet, brand_parser)
print(len(brandKB))


# start at 1559909368.03804
# end at 1559909368.04514
total cost: 0.00709
6


In [109]:
def kw_match(content, keywords):
    """
    关键词匹配
    """
    for kw in keywords:
        if kw in content:
            return True
    return False

def brand_match(content, kb):
    """
    品牌匹配
    """
    for key, value in kb.items():
        if kw_match(content, value):
            return key
    
    return None

# TEST CASE

print(brand_match("vivo", brandKB))
print(brand_match("#我才是实力自拍王", brandKB))
print(brand_match("vivo", brandKB))
print(brand_match("#华为", brandKB))

VIVO
华为
VIVO
None


In [46]:
import time

def common_time_parse(origin):
    """
    时间处理
    """
    real_time = time.localtime() # 默认当前时间
    if type(origin) is str and len(origin) == 19:
        real_time = time.strptime(origin, "%Y-%m-%d %H:%M:%S")
    elif len(str(origin)) == 14:
        real_time = time.strptime(str(origin), "%Y%m%d%H%M%S")

    return real_time

# time.strftime(real_time, "%Y-%m-%d %H:%M:%S")

In [63]:
video_tags = {}

def video_parser(idx, key, row):
    video_tag = {}
    
    url = row['url']
    vid = url.split('/')[5]
    uid = row["user_item_id"]
    
    publish_date = time.strftime("%Y-%m-%d %H:%M:%S", common_time_parse(row["publish_date"]))
    title = row["title"]
    content = row['content']
    brand = ""
    if type(content) is str:
        brand = brand_match(content, brandKB)
        
    video_tag["vid"] = vid
    video_tag["url"] = url
    video_tag["publish_date"] = publish_date
    video_tag["title"] = title
    video_tag["content"] = content
    video_tag["brand"] = brand
       
    user_tag = {}
    if uid in user_tags:
        user_tag = user_tags[uid]
    else:
        print("user not exist: %s" % uid)
    for k, v in user_tag.items():
        video_tag["user_" + k] = v

    video_tags[vid] = video_tag
    return True
    
each_row(video_sheet, video_parser, 1000)
print(len(video_tags))

# start at 1559907477.89940
user not exist: 4173365255879406
# end at 1559907478.07784
total cost: 0.17844
760


In [66]:
print(len(video_tags))
for k, v in video_tags.items():
    print(k)
    print(v)
    break

760
6641823319651781896
{'vid': '6641823319651781896', 'url': 'https://www.iesdouyin.com/share/video/6641823319651781896/?region=CN&mid=6641823330779269902&u_code=jdajj3e5&titleType=title', 'publish_date': '2019-01-02 17:06:39', 'title': '@海飞数码创作的原声', 'content': '目前拍照最好的十部手机，让我们大喊一句“国产牛逼”#手机 #华为 #欢脱喜提开门红', 'brand': None, 'user_uid': 74968768187, 'user_name': '海飞数码', 'user_fans': 1308577, 'user_register_id': '118900302'}


## 评论处理



In [110]:
comment_file = 'analysis_task_20190606230755_761_93.xlsx'
comment_all_sheet = read_excel(abs_path + comment_file)
print(len(comment_all_sheet))

# start at 1559909400.11760
predict cost: 333.00000s
# end at 1559909690.41367
total cost: 290.29607
3


> 安全隔离带

In [123]:
current_sheet = "sheet0"

In [127]:
comment_test_sheet = comment_all_sheet[current_sheet]
print(comment_test_sheet.shape)

comment_tags = {}
comment_infos = {}
def comment_parser(idx, key, row):
    comment_tag = {}
    
    cid = row["comment_id"]
    url = row["url"]
    vid = url.split('/')[5]
    
    publish_date = time.strftime("%Y-%m-%d %H:%M:%S", common_time_parse(row["publish_date"]))
    
    comment_tag["new_publish_date"] = publish_date
    comment_tag["new_vid"] = vid
    
    video_tag = {}
    if vid in video_tags:
        video_tag = video_tags[vid]
    else:
        print("video not exist: %d" % vid)
    for k, v in video_tag.items():
        comment_tag["video_" + k] = v
        
#     comment_tags[cid] = comment_tag
#     comment_infos[cid] = row
    
    return True
    
each_row(comment_test_sheet, comment_parser, 5000)

print(len(comment_tags))
# print(comment_tags)
print(len(comment_infos))
# print(comment_infos)

# chunksize=4, csv才有

(500000, 23)
# start at 1559911703.19655
current time: 1559911705.338710
current idx: 5000
current time: 1559911705.820023
current idx: 10000
current time: 1559911706.284814
current idx: 15000
current time: 1559911706.750563
current idx: 20000
current time: 1559911707.215227
current idx: 25000
current time: 1559911707.727630
current idx: 30000
current time: 1559911708.203805
current idx: 35000
current time: 1559911708.730173
current idx: 40000
current time: 1559911709.242209
current idx: 45000
current time: 1559911709.722802
current idx: 50000
current time: 1559911710.251229
current idx: 55000
current time: 1559911710.773056
current idx: 60000
current time: 1559911711.291025
current idx: 65000
current time: 1559911711.794749
current idx: 70000
current time: 1559911712.286279
current idx: 75000
current time: 1559911712.767485
current idx: 80000
current time: 1559911713.241447
current idx: 85000
current time: 1559911713.693356
current idx: 90000
current time: 1559911714.150389
current id

In [ ]:
print(len(comment_tags))
for k, v in comment_tags.items():
    print(k)
    print(v)
    break
print(len(comment_infos))
for k, v in comment_infos.itmes():
    print(k)
    print(v)
    break

In [ ]:
comment_infos_df = pd.DataFrame(comment_infos)
comment_tags_df = pd.DataFrame(comment_tags)

comment_df = pd.concat([comment_infos_df, comment_tags_df], axis=0)

print(comment_infos_df.shape)
print(comment_tags_df.shape)
print(comment_df.shape)

In [ ]:
save_excel(comment_df.T, abs_path + "comment-" + current_sheet + ".xlsx")